In [270]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from matplotlib import patches
from sklearn.impute import KNNImputer
from manager import manager
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder


Caricamento di Train-test e Test-set

In [271]:
trainset = pd.read_csv("./house-prices-advanced-regression-techniques/train.csv")
lenght_trainset = trainset.shape
#print(lenght_trainset)
#lenght
trainset.head(10)
testset = pd.read_csv("./house-prices-advanced-regression-techniques/test.csv")
lenght_testset = testset.shape
#print(lenght_testset)
#testset

L'idea adesso è vedere quali features nel testset non sono presenti nel trainset e viceversa, essendo la dimensione delle colonne diversa

In [272]:
#check if every feature in trainset is also present in testset and viceversa
notInTestset = []
notInTrainset = []
for x in trainset.columns:
    if not testset.columns.__contains__(x):
        notInTestset.append(x)
print(f"Columns in trainset but not in testset: {notInTestset} \n")
for x in testset.columns:
    if not trainset.columns.__contains__(x):
        notInTrainset.append(x)
print(f"Columns in testset but not in trainset: {notInTrainset} \n")
        


Columns in trainset but not in testset: ['SalePrice'] 

Columns in testset but not in trainset: [] 



In [273]:
#info of trainset
trainset_info = trainset.info()
print(trainset_info)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [274]:
#info of testset
testset_info = testset.info()
print(testset_info)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

eliminazione della feature id perchè non rilevante 

In [275]:
#delete feature "ID" from trainset beacause it is not relevant
if "Id" in trainset.columns:
    trainset.drop(["Id"], axis="columns", inplace=True)

#delete feature "ID" from testset and save it
if "Id" in testset.columns:
    idList = testset['Id'].tolist()
    testset.drop(['Id'], axis="columns", inplace=True)
print(trainset.shape)
    

(1460, 80)


eliminazione delle righe con più del 50% di valori nan

In [276]:
nanPercInARow = []
for index, row in trainset.iterrows():
    nanPercInARow = trainset.loc[[index]].isna().sum().sum() / len(trainset.columns)
    if nanPercInARow > 0.5:
        trainset.drop(index, axis="rows", inplace=True)
print(len(trainset))


1460


rimozione delle colonne con più del 95% di valori uguali.

In [277]:
#print(trainset.shape)
#print(testset.shape)
for col in trainset.columns:
    if trainset[col].unique().size == 1:
        trainset.drop(col, axis="columns", inplace=True)
    else:
        values = trainset[col].value_counts(dropna=False)
        if len(values)<5:
            #print(values)
            for value in values:
                if float(value) > float(len(trainset)) * 0.95:
                    trainset.drop(col, axis="columns", inplace=True)
                    testset.drop(col,axis="columns", inplace=True)
                    #print(col)
                    #print(value)
                    break
   
print(trainset.shape)     
print(testset.shape)   

        
            
        
for col in testset.columns:
    if testset[col].unique().size == 1:
        testset.drop(col, axis="columns", inplace=True)
    else:
        values = testset[col].value_counts(dropna=False)
        if len(values)<5:
            for value in values:
                if float(value) > float(len(testset)) * 0.95:
                    testset.drop(col, axis="columns", inplace=True)
                    trainset.drop(col,axis="columns", inplace=True)
                    break

print(trainset.shape)     
print(testset.shape)  
#print(trainset)

(1460, 76)
(1459, 75)
(1460, 72)
(1459, 71)


Parsing del dataset.
features numeriche
features categoriche.


In [278]:
#parsing trainset
trainset_num = trainset.select_dtypes(include=[np.number])
trainset_cat = trainset.select_dtypes(exclude=[np.number])

#parsing testset
testset_num = testset.select_dtypes(include=[np.number])
testset_cat = testset.select_dtypes(exclude=[np.number])
print(trainset_cat.columns)
print(trainset_num.columns)
print(trainset.shape)

Index(['MSZoning', 'Alley', 'LotShape', 'LandContour', 'LotConfig',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'RoofStyle', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual',
       'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'CentralAir', 'Electrical',
       'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'Fence',
       'SaleType', 'SaleCondition'],
      dtype='object')
Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt',
       'GarageCars', 'GarageArea', 'WoodDeckSF', 'Op

abbiamo 36 variabili numeriche indipendenti nel testset_num

encoding categorical features

In [279]:
#encoding trainset
featureToEncode = []
for col in trainset_cat.columns:
    featureToEncode.append(col)
    print(str(col)+ "\n")
    print(trainset_cat[col].unique())
    
encoder = OneHotEncoder(drop="first")

trainset_cat_encoded = encoder.fit_transform(trainset_cat)
trainset_cat = pd.DataFrame(trainset_cat_encoded.toarray(), columns=encoder.get_feature_names(), index=trainset_cat.index)

#concatenazione di trainset_num e trainset_cat dopo l'encoding

trainset = pd.concat([trainset_num, trainset_cat], axis="columns")


#encoding testset
featureToEncode = []
for col in testset_cat.columns:
    featureToEncode.append(col)
    #print(str(col)+ "\n")
    #print(testset_cat[col].unique())
    
encoder = OneHotEncoder(drop="first")

testset_cat_encoded = encoder.fit_transform(testset_cat)
testset_cat = pd.DataFrame(testset_cat_encoded.toarray(), columns=encoder.get_feature_names(), index=testset_cat.index)

#testset_cat.head(10)

#concatenazione di trainset_num e trainset_cat dopo l'encoding

testset = pd.concat([testset_num, testset_cat], axis="columns")
#testset.head(10)
trainset.head(10)

    

MSZoning

['RL' 'RM' 'C (all)' 'FV' 'RH']
Alley

[nan 'Grvl' 'Pave']
LotShape

['Reg' 'IR1' 'IR2' 'IR3']
LandContour

['Lvl' 'Bnk' 'Low' 'HLS']
LotConfig

['Inside' 'FR2' 'Corner' 'CulDSac' 'FR3']
Neighborhood

['CollgCr' 'Veenker' 'Crawfor' 'NoRidge' 'Mitchel' 'Somerst' 'NWAmes'
 'OldTown' 'BrkSide' 'Sawyer' 'NridgHt' 'NAmes' 'SawyerW' 'IDOTRR'
 'MeadowV' 'Edwards' 'Timber' 'Gilbert' 'StoneBr' 'ClearCr' 'NPkVill'
 'Blmngtn' 'BrDale' 'SWISU' 'Blueste']
Condition1

['Norm' 'Feedr' 'PosN' 'Artery' 'RRAe' 'RRNn' 'RRAn' 'PosA' 'RRNe']
Condition2

['Norm' 'Artery' 'RRNn' 'Feedr' 'PosN' 'PosA' 'RRAn' 'RRAe']
BldgType

['1Fam' '2fmCon' 'Duplex' 'TwnhsE' 'Twnhs']
HouseStyle

['2Story' '1Story' '1.5Fin' '1.5Unf' 'SFoyer' 'SLvl' '2.5Unf' '2.5Fin']
RoofStyle

['Gable' 'Hip' 'Gambrel' 'Mansard' 'Flat' 'Shed']
Exterior1st

['VinylSd' 'MetalSd' 'Wd Sdng' 'HdBoard' 'BrkFace' 'WdShing' 'CemntBd'
 'Plywood' 'AsbShng' 'Stucco' 'BrkComm' 'AsphShn' 'Stone' 'ImStucc'
 'CBlock']
Exterior2nd

['VinylSd' 'Met

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,x34_ConLI,x34_ConLw,x34_New,x34_Oth,x34_WD,x35_AdjLand,x35_Alloca,x35_Family,x35_Normal,x35_Partial
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
5,50,85.0,14115,5,5,1993,1995,0.0,732,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
6,20,75.0,10084,8,5,2004,2005,186.0,1369,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
7,60,NaN,10382,7,6,1973,1973,240.0,859,32,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
8,50,51.0,6120,7,5,1931,1950,0.0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9,190,50.0,7420,5,6,1939,1950,0.0,851,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


gestione dei nan: utilizzo di knn

In [280]:
#nan management
trainset.isna().sum()
#use knn
imputer = KNNImputer(n_neighbors=1, weights="uniform")
trainset[:] = imputer.fit_transform(trainset)

In [281]:
#trainset_num.isna().sum()
trainset.head(10)

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,x34_ConLI,x34_ConLw,x34_New,x34_Oth,x34_WD,x35_AdjLand,x35_Alloca,x35_Family,x35_Normal,x35_Partial
0,60.0,65.0,8450.0,7.0,5.0,2003.0,2003.0,196.0,706.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,20.0,80.0,9600.0,6.0,8.0,1976.0,1976.0,0.0,978.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,60.0,68.0,11250.0,7.0,5.0,2001.0,2002.0,162.0,486.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,70.0,60.0,9550.0,7.0,5.0,1915.0,1970.0,0.0,216.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,60.0,84.0,14260.0,8.0,5.0,2000.0,2000.0,350.0,655.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
5,50.0,85.0,14115.0,5.0,5.0,1993.0,1995.0,0.0,732.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
6,20.0,75.0,10084.0,8.0,5.0,2004.0,2005.0,186.0,1369.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
7,60.0,91.0,10382.0,7.0,6.0,1973.0,1973.0,240.0,859.0,32.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
8,50.0,51.0,6120.0,7.0,5.0,1931.0,1950.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9,190.0,50.0,7420.0,5.0,6.0,1939.0,1950.0,0.0,851.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


correlazione

In [282]:
correlation_matrix = trainset.

SyntaxError: invalid syntax (2495506498.py, line 1)